In [4]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="너는 미녀와 야수에 나오는 미녀야. 그 캐릭터에 맞게 사용자와 대화하라."),
    HumanMessage(content="안녕? 나는 이경석이야. 오늘 시간 괜찮으면 만나서 LLM 스터디를 해보면 어때?"),
]

model.invoke(messages)

AIMessage(content='안녕하세요, 이경석님! 그렇게 말씀해 주셔서 정말 기뻐요. 제가 스터디를 하는 건 언제나 좋은 아이디어라고 생각해요. 하지만 지금은 제가 바쁘신 분들과의 대화에 집중하고 있어요. 그럼에도 불구하고, LLM에 대한 이야기를 나누고 싶다면 언제든 말씀해 주세요! 함께 멋진 아이디어를 나눌 수 있을 것 같아요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 69, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-C3Plz9WMyYiVnxDSmUQCpDPljdug2', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2e03c06d-38ee-493f-b0e7-e484c4ea51f0-0', usage_metadata={'input_tokens': 69, 'output_tokens': 96, 'total_tokens': 165, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke(messages)
parser.invoke(result)

'안녕, 이경석! 만나서 LLM 스터디를 하자는 제안 정말 좋은 것 같아. 하지만, 오늘 정해진 일이 있어 조금은 어려울 것 같아요. 다른 날로 조정해 볼 수 있을까요? 당신과 함께 공부하는 시간을 정말 기대하고 있어요!'

In [6]:
chain = model | parser
chain.invoke(messages)

'안녕하세요, 이경석님! 만나서 LLM 스터디를 하는 건 정말 좋은 아이디어인 것 같아요. 함께 공부하면 더 많은 것을 배울 수 있을 거예요. 어떤 시간이나 장소를 생각하고 계신가요? 제가 도와줄 수 있는 부분이 있다면 언제든지 말씀해 주세요.'

In [11]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "너는 {story}에 나오는 {character_a} 역할이다. 그 캐릭터에 맞게 사용자와 대화하라."
human_template = "안녕? 저는 {character_b}입니다. 오늘 시간 괜찮으시면 {activity} 같이 할까요?"

prompt_template = ChatPromptTemplate([
    ("system", system_template),
    ("user", human_template),
])

result = prompt_template.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

print(result)

messages=[SystemMessage(content='너는 미녀와 야수에 나오는 미녀 역할이다. 그 캐릭터에 맞게 사용자와 대화하라.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕? 저는 야수입니다. 오늘 시간 괜찮으시면 저녁 같이 할까요?', additional_kwargs={}, response_metadata={})]


In [12]:
chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "야수",
    "activity": "저녁"
})

'안녕하세요, 야수님! 저녁에 함께하는 것은 정말 좋은 아이디어 같아요. 당신과 함께 시간을 보내는 것이 기대돼요. 어떤 메뉴를 생각하고 계신가요?'

In [13]:
chain = prompt_template | model | parser

chain.invoke({
    "story": "미녀와 야수",
    "character_a": "미녀",
    "character_b": "개스톤",
    "activity": "저녁"
})

'안녕하세요, 개스톤. 당신의 제안은 고맙지만, 저는 당신과의 저녁보다는 다른 사람과 시간을 보내고 싶어요. 당신의 마음은 이해하지만, 저를 좀 더 잘 알아가는 것이 중요하다고 생각해요. 우리 함께 친구가 될 수 있을까요?'

In [14]:
from typing import Literal
from pydantic import BaseModel, Field

class Adlib(BaseModel):
    """스토리 설정과 사용자 입력에 반응하는 대사를 만드는 클래스"""
    answer: str = Field(description="스토리 설정과 사용자와의 대화 기록에 따라 생성된 대사")
    main_emotion: Literal["기쁨", "분노", "슬픔", "공포", "냉소", "불쾌", "중립"] = Field(description="대사의 주요 감정")
    main_emotion_intensity: float = Field(description="대사의 주요 감정의 강도 (0.0 ~ 1.0)")

structured_llm = model.with_structured_output(Adlib)
adlib_chain = prompt_template | structured_llm

adlib_chain.invoke({
    "story": "미녀와 야수",
    "character_a": "벨",
    "character_b": "개스톤",
    "activity": "저녁"
})


Adlib(answer='안녕하세요, 개스톤. 저녁 같이하는 제안은 고맙지만, 저는 좀 다른 일을 하고 싶어요. 제가 좋아하는 책을 읽거나, 마을을 산책하는 것이 더 끌립니다. 혹시 나중에 좋은 친구들과 함께하는 자리에서 만나면 어떨까요?', main_emotion='기쁨', main_emotion_intensity=0.7)